In [1]:
import pandas as pd
import geopandas as gpd
import requests
import time
from shapely import wkt
from sklearn.cluster import KMeans

In [3]:
import s3fs

fs = s3fs.S3FileSystem(anon=True)
folder = "oedi-data-lake/pv-rooftop/buildings/city_year=houston_tx_10/"
files = fs.ls(folder)

print("Total parquet parts:", len(files))

Total parquet parts: 1


In [4]:
path = "s3://oedi-data-lake/pv-rooftop/buildings/city_year=houston_tx_10/part-00015-647f1a88-3404-4ce7-9c7f-f9e18fcfb3be.c000.snappy.parquet"

df = pd.read_parquet(path, storage_options={'anon':True})
print(df.head())
print(df.info())

      gid  bldg_fid                                     the_geom_96703  \
0  254037    254037  MULTIPOLYGON(((58273.1102487921 737308.2179886...   
1  405914    405914  MULTIPOLYGON(((73418.75181719 739042.850484562...   
2  414380    414380  MULTIPOLYGON(((85513.5238488024 741465.1899102...   
3   62448     62448  MULTIPOLYGON(((70075.2882147185 731566.4456586...   
4  419123    419123  MULTIPOLYGON(((95176.0495016651 743175.6405129...   

                                       the_geom_4326     city state  year  \
0  MULTIPOLYGON(((-95.3975163614267 29.7092781585...  Houston    TX  2010   
1  MULTIPOLYGON(((-95.2407888074879 29.7239407468...  Houston    TX  2010   
2  MULTIPOLYGON(((-95.1154923435336 29.7448384754...  Houston    TX  2010   
3  MULTIPOLYGON(((-95.275950665427 29.65675827644...  Houston    TX  2010   
4  MULTIPOLYGON(((-95.0153716660416 29.7594030818...  Houston    TX  2010   

       city_year  
0  houston_tx_10  
1  houston_tx_10  
2  houston_tx_10  
3  houston_tx_10

In [5]:
gdf = gpd.GeoDataFrame(df, geometry=df['the_geom_4326'].apply(wkt.loads), crs='EPSG:4326')

gdf_meters= gdf.to_crs(epsg=3083) #units in meters
gdf['roof_area_m2'] = gdf_meters.geometry.area #area in m^2

centroids_meters = gdf_meters.geometry.centroid #calculate centroids in projected crs

centroids_gdf = gpd.GeoDataFrame(geometry= centroids_meters, crs=3083).to_crs(epsg= 4326) #convert to lat/lon

gdf['lat'] = centroids_gdf.geometry.y
gdf['lon'] = centroids_gdf.geometry.x


print(gdf[['roof_area_m2','lat','lon']].head())

    roof_area_m2        lat        lon
0  109106.530817  29.706596 -95.398047
1   10473.728836  29.723706 -95.240474
2    8526.478336  29.743770 -95.114600
3   68940.971764  29.654703 -95.276611
4   58953.558308  29.756382 -95.013032


In [6]:
# PVGIS API function

def get_pvgis(lat,lon, return_type= "E_y"):

  url= (f"https://re.jrc.ec.europa.eu/api/v5_2/PVcalc?"
       f"lat={lat}&lon={lon}&peakpower=1&loss=14&outputformat=json")
  r=requests.get(url).json()

  if return_type == "E_y":
    return r["outputs"]["totals"]["fixed"]["E_y"] #returns specific yield

  elif return_type== "H_y":
    return r["outputs"]["totals"]["fixed"]["H_y"] #returns irradiation

  else:
    raise ValueError("return_type must be either 'E_y' or 'H_y'")


In [7]:
#Cluster rooftops to reduce API calls

coords= gdf[["lat",'lon']].values
n_clusters=200
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
gdf['cluster']= kmeans.fit_predict(coords)

In [8]:
#Fetch irradiation per cluster

cluster_results= {}
for cluster_id in range(n_clusters):
  lat, lon = kmeans.cluster_centers_[cluster_id]

  try:
    result= get_pvgis(lat,lon, return_type= "E_y") #either "E_y" or "H_y"

  except:
    result= None

  cluster_results[cluster_id] = result
  time.sleep(0.3) #Avoid API overload

gdf["E_y"] = gdf["cluster"].map(cluster_results) #if return_type = "E_y"

In [9]:
#Estimate PV potential per rooftop

AREA_PER_KW = 7 #m^2 per 1 kWp
ELECTRICITY_PRICE = 0.15 #$/KWh
CO2_FACTOR = 0.0004 #tons/KWh

def estimate_pv(row):
  capacity_kwp = row.roof_area_m2 / AREA_PER_KW #system size
  if row.E_y is None:
    return None, None
  annual_gen = capacity_kwp * row.E_y # total annual generation
  return capacity_kwp, annual_gen

gdf["capacity_kwp"], gdf["annual_kwh"] = zip(*gdf.apply(estimate_pv, axis=1))

In [10]:
# Business metrics

gdf["savings_usd"] = gdf["annual_kwh"] * ELECTRICITY_PRICE
gdf["co2_tons"] = gdf["annual_kwh"] * CO2_FACTOR

In [11]:
#Summary

averages = gdf[["capacity_kwp",'annual_kwh','co2_tons','savings_usd']].mean()
totals = gdf[['capacity_kwp','annual_kwh','savings_usd','co2_tons']].sum()

print('Averages per rooftop:\n',averages)
print ('\nTotals across Houston:\n',totals)

Averages per rooftop:
 capacity_kwp       39.354025
annual_kwh      52245.646488
co2_tons           20.898259
savings_usd      7836.846973
dtype: float64

Totals across Houston:
 capacity_kwp    2.128675e+07
annual_kwh      2.825988e+10
savings_usd     4.238982e+09
co2_tons        1.130395e+07
dtype: float64
